<a href="https://colab.research.google.com/github/sudevansujit/Movie_Sentiment_Analysis/blob/master/Movie_Sentiment_Analysis_Classic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
downloaded = drive.CreateFile({'id':'1roXwO-PkQPpN8HkrWo5gBpFrz3vh2vo5'}) # replace the id with id of file you want to access
downloaded.GetContentFile('movie_review.csv')

In [ ]:
downloaded = drive.CreateFile({'id':'1mBFPNOxy_O4mh8DWPD_4FNzDXmpJ0fn-'}) # replace the id with id of file you want to access
downloaded.GetContentFile('slang.csv')

In [4]:
import pandas as pd
import numpy as np

data = pd.read_csv('movie_review.csv', index_col = 'Unnamed: 0')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
slang = pd.read_csv('slang.csv', index_col = 'Unnamed: 0')
slang.head()

,a-town,a.o.e.,a/n,a3,a7x,aa,aaa,aaftddup,aanfctw,aapl,aatk,ab,abb,abc,abot,acgad,ad,adad,adidas,adiem,afaik,afap,afk,afkst,afm,afol,afp,afret,ahic,ahz,aiiegaf,aijcomkb,ain't,alaittrwl,alwas,amazin,anof,ansx,ao,aom,...,yswnt,yt,ytd,ytf,ytfwudt,ythwudt,ytis,ytm,ytmnd,yty,yu,yua,yuo,yup,yur,yust,yvfw,yvw,yw,ywapom,ywia,ywic,yws,ywsyls,ywud,ywvm,ywywm,yysw,z'omg,z0mg,zex,zh,zig,zomfg,zomg,zomgzorrz,zoot,zot,zt,zup
a,"atlanta,ga",area of effect,authors note,"anytime, anywhere, anyplace",avenged sevenfold,allahu akbar,"anywhere, any place, any time",as a friend till death do us part,a**h**es are not f**king checking these words,apple computer inc.,asleep at the keyboard,about,anything but bush,already been chewed,about,absolutely couldn't give a damn,advertisement,another day another doller,all day i dream about sex,all day i eat meat,as far as i know,as far as possible,away from keyboard,away from keyboard stop talking,away from mouse,adult fan of lego,away from phone,air force retired,ah i see,ahz,as if i even give a f**k,"aw, i just came on my keyboard",is not,at least add it to the rejected words list.,always,amazing,enough,anal sex,auto op irc,age of mythology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data['sentiment'] = data['sentiment'].map({'positive' : 1, 'negative' : 0})
data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [8]:
# Removing html tags
from bs4 import BeautifulSoup
def remove_html(text):
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    return text

data['review'] = data['review'].apply(remove_html)
data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [9]:
# Remove all URLs
def remove_urls(text):
    text = re.sub(r'https?:\/\/\S*| www\.\S*', '', text, flags=re.MULTILINE)
    return text

import re
data['review'] = data['review'].apply(remove_urls)
data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [10]:
# Remove all # and @ words
def remove_sym_words(text):
    text = re.sub(r'@\w*\S| #\w*\S', ' ', text)
    return text

data['review'] = data['review'].apply(remove_sym_words)
data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [11]:
# Remove Accented
import unicodedata

def remove_accented(text):
    text = unicodedata.normalize("NFKD", text ).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

data['review'] = data['review'].apply(remove_accented)
data.head()                
 

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [12]:
              
# Replacing Punctuation
data['review'] = data['review'].str.replace("`", "'")
data.head()                
 

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [13]:
# Removing Numbers
def remove_num(text):
    text = re.sub(r'\d*', '', text)
    return text                
                
data['review'] = data['review'].apply(remove_num)
data.head() 


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [14]:
# Expanding contractions

slang_columns = list(slang.columns)

def replace_slang(text):
    text = text.lower()
    mylis = text.split()
    new = []
    for i in mylis:
        if isinstance(i, str):
            if i in slang_columns:
                new.append(slang[i].loc[i[0]])
            else:
                new.append(i)
        else:
            continue
    text = ' '.join(new)
    return text

data['new_review'] = data['review'].apply(replace_slang)
data.head(5)


,review,sentiment,new_review
0,One of the other reviewers has mentioned that ...,1,one of the other reviewers has mentioned that ...
1,A wonderful little production. The filming tec...,1,a wonderful little production. the filming tec...
2,I thought this was a wonderful way to spend ti...,1,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,0,basically there is a family where a little boy...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"petter mattei's ""love in the time of money"" is..."


In [15]:
data['review'][2]


'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point : Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

In [16]:
data['new_review'][2]


'i thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. the plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). while some may be disappointed when they realize this is not match point : risk addiction, i thought it was proof that woody allen is still fully in control of the style many of us have grown to love.this was the most i would laughed at one of woody\'s comedies in years (dare i say a decade?). while i have never been impressed with scarlet johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.this may not be the crown jewel of his career, but it was wittier than "devil wears prada" and more interesting than "superman" a great comedy to go see with friends.'

In [18]:
# Remove Stopwords
import nltk
nltk.download('stopwords')
import string
letters = string.ascii_letters
small_alpha = list(letters[:26])
small_alpha

from nltk.corpus import stopwords
stop = stopwords.words('english')

mystop = stop + small_alpha

def remove_stop_words(text):
    text = [word  for word in text.split() if word not in mystop]
    text = ' '.join(text)
    return text

data['new_review'] = data['new_review'].apply(remove_stop_words)
data.head() 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,review,sentiment,new_review
0,One of the other reviewers has mentioned that ...,1,one reviewers mentioned watching oz episode ho...
1,A wonderful little production. The filming tec...,1,wonderful little production. filming technique...
2,I thought this was a wonderful way to spend ti...,1,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,0,basically family little boy (jake) thinks zomb...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"petter mattei's ""love time money"" visually stu..."


In [19]:
# Clean Text

def clean(text):
    
    mylist = [' ' if alpha in list(string.punctuation) else alpha for alpha in text ]
    text = ''.join(mylist)
    text = text.replace('  ', ' ')
    text = text.replace('  ', ' ')
    text = text.lower().strip()
    return text

data['new_review'] = data['new_review'].apply(clean)
data.head() 


,review,sentiment,new_review
0,One of the other reviewers has mentioned that ...,1,one reviewers mentioned watching oz episode ho...
1,A wonderful little production. The filming tec...,1,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,1,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,0,basically family little boy jake thinks zombie...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter mattei s love time money visually stunn...


In [20]:
data['review'] = data['review'].apply(clean)
data.head() 


,review,sentiment,new_review
0,one of the other reviewers has mentioned that ...,1,one reviewers mentioned watching oz episode ho...
1,a wonderful little production the filming tech...,1,wonderful little production filming technique ...
2,i thought this was a wonderful way to spend ti...,1,thought wonderful way spend time hot summer we...
3,basically there s a family where a little boy ...,0,basically family little boy jake thinks zombie...
4,petter mattei s love in the time of money is a...,1,petter mattei s love time money visually stunn...


In [21]:
data['new_review'][2]


'thought wonderful way spend time hot summer weekend sitting air conditioned theater watching light hearted comedy plot simplistic dialogue witty characters likable even well bread suspected serial killer may disappointed realize match point risk addiction thought proof woody allen still fully control style many us grown love this would laughed one woody s comedies years dare say decade never impressed scarlet johanson managed tone sexy image jumped right average spirited young woman this may crown jewel career wittier devil wears prada interesting superman great comedy go see friends'

In [22]:

data['review'][2]


'i thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a light hearted comedy the plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer while some may be disappointed when they realize this is not match point risk addiction i thought it was proof that woody allen is still fully in control of the style many of us have grown to love this was the most i d laughed at one of woody s comedies in years dare i say a decade while i ve never been impressed with scarlet johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young woman this may not be the crown jewel of his career but it was wittier than devil wears prada and more interesting than superman a great comedy to go see with friends'

In [ ]:
#data.to_csv('Cleaned_Movie_Review.csv')


In [23]:
# build train and test datasets
X_review = data['new_review'].values
y_sentiment = data['sentiment'].values

X_train = X_review[:35000]
y_train = y_sentiment[:35000]

X_test = X_review[35000:]
y_test = y_sentiment[35000:]

X_train.shape, y_train.shape


((35000,), (35000,))

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range = (1, 2), 
                        max_features = 50000, 
                        min_df=5, 
                        max_df=1.0,
                        use_idf = True)
tfidf_train = tfidf.fit_transform(X_train)         # 35000, 50000
tfidf_test = tfidf.transform(X_test)              # 15000, 50000


In [25]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l2', max_iter=500, C=1, solver='lbfgs', random_state=42 )
lr_model = lr.fit(tfidf_train, y_train)
lr_model


y_test.reshape(-1, 1 ).shape

y_pred = lr_model.predict(tfidf_test)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print("% Accuracy = ",accuracy_score(y_test, y_pred)*100)

print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))


% Accuracy =  90.32666666666667
[[6697  793]
 [ 658 6852]]
              precision    recall  f1-score   support

           0       0.91      0.89      0.90      7490
           1       0.90      0.91      0.90      7510

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



In [26]:
# Random Forest model on BOW features
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
# train model
rf.fit(tfidf_train, y_train)
# predict on test data
y_pred = rf.predict(tfidf_test)


print("% Accuracy = ",accuracy_score(y_test, y_pred)*100)

print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))


% Accuracy =  86.11333333333333
[[6536  954]
 [1129 6381]]
              precision    recall  f1-score   support

           0       0.85      0.87      0.86      7490
           1       0.87      0.85      0.86      7510

    accuracy                           0.86     15000
   macro avg       0.86      0.86      0.86     15000
weighted avg       0.86      0.86      0.86     15000



In [27]:
#Thanks